# FC25 Rush Detection Model
## Deploy as a Service
This notebook will deploy the Rush detection model as a service. The goal is to leverage as much project code as possible, and to keep this notebook as lightweight as possible.

Ideally, this notebook will be run on a local subnet, and the service will be accessed via a local IP address. Alternatively, it could be run on colab, and accessed via a public address.

### Setup
From your colab notebook, you should be able to run the following commands to setup the environment. Don't forget to create a `.env` file in the `src` directory with your API keys.
`HF_API_KEY`
`HF_RUSH_DETECTION_PATH`
`HF_RUSH_DETECTION_FILENAME`
`NGROK_AUTH_TOKEN`
```
%%capture
%%sh
git clone https://github.com/swaynos/urban-palm-tree.git

%cd urban-palm-tree/src
```

In [ ]:
import sys
import os

# Check if the current working directory is under 'notebooks'
if 'notebooks' in os.getcwd().split(os.sep):
    # Adjust the path to include the 'src' directory
    sys.path.append(os.path.abspath(os.path.join('..', 'src')))  # This adds the correct src path

# Verify the current system path
print(sys.path)

## Run the Service from Jupyter

### Step 1. Environment Setup
Install the required packages.

In [ ]:
%%capture
%%sh
pip install aiofiles dotenv fastapi huggingface_hub python-multipart uvicorn pillow numpy gunicorn scikit-image torch ultralytics

### Step 2. 
The endpoint code.

### Step 3. Run the Service

In [ ]:
# Run Uvicorn in a background thread
import threading
server_thread = threading.Thread(target=lambda: uvicorn.run(app, host="0.0.0.0", port=6969), daemon=True)
server_thread.start()


#### Running the Service from Google Colab

In [ ]:
# If running on Colab
!pip install pyngrok
from pyngrok import ngrok
import os

# Authenticate ngrok with your token
NGROK = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK)

# Define a function to start Flask
def run_app():
    os.system(f"python3 -m flask run --host=0.0.0.0 --port={PORT}")

# Start Flask in the background
import threading
threading.Thread(target=run_app).start()

# Step 5: Set up ngrok tunnel
public_url = ngrok.connect(PORT)
print(f"ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{PORT}\"")
